### `rabi-split` Data Viewer
#### Rev. 8/23/2022.

In [ ]:
import au
import h5py
import numpy as np
import scipy.constants as mks

import matplotlib
import matplotlib.pyplot as plt

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

# data directory
home_path = '/media/ayounis/DATA1/rabi-split'
run_path = 'run23'

### Read input deck

In [ ]:
f = open(home_path+'/'+run_path+'/'+'input.deck','r')
for ln in f:
    # wavefunction domain
    if (ln[0:5] == 't_max'): sln = ln.split(); t_max = eval(sln[1].replace('d','e')); # simulation time
    if (ln[0:5] == 'r_max'): sln = ln.split(); r_max = eval(sln[1].replace('d','e')); # radial extent
    if (ln[0:5] == 'l_max'): sln = ln.split(); l_max = int(sln[1]); # OAM numbers
    if (ln[0:2] == 'dr'): sln = ln.split(); dr = eval(sln[1].replace('d','e')); # spatial resolution
    
    # atomic properties
    if (ln[0:3] == 'nlm'): sln = ln.split(); enn = int(sln[1]); ell = int(sln[2]); emm = int(sln[3]); # initial-state (nlm)
    
    # p-electron spectrum
    if (ln[0:3] == 'rng'):
        sln = ln.split()
        e_min = eval(sln[1].replace('d','e')) # spectrum range
        e_max = eval(sln[2].replace('d','e'))
    if (ln[0:3] == 'res'): sln = ln.split(); de = eval(sln[1].replace('d','e')); # spectrum resolution
    if (ln[0:3] == 'ord'): sln = ln.split(); nwop = int(sln[1]); # spectrum order
    
    # field
    if (ln[0:2] == 'Eu'): sln = ln.split(); Fu = eval(sln[1].replace('d','e')); # pump/probe strength
    if (ln[0:2] == 'Er'): sln = ln.split(); Fr = eval(sln[1].replace('d','e'));
    
    if (ln[0:5] == 'omg_u'): sln = ln.split(); omg_u = eval(sln[1].replace('d','e')); # pump/probe frequency
    if (ln[0:5] == 'omg_r'): sln = ln.split(); omg_r = eval(sln[1].replace('d','e'));
    
    if (ln[0:5] == 'tu_on'): sln = ln.split(); tu_on = eval(sln[1].replace('d','e')); # pump/probe start time
    if (ln[0:5] == 'tr_on'): sln = ln.split(); tr_on = eval(sln[1].replace('d','e'));

# atomic transition frequencies
omg21 = -0.5*(1/4-1)
omg31 = -0.5*(1/9-1)
omg32 = -0.5*(1/9-1/4)

# laser periods
T0u = 2*np.pi/omg_u
T0r = 2*np.pi/omg_r

# ponderomotive energies
Upu = (Fu**2)/4.0/(omg_u**2)
Upr = (Fr**2)/4.0/(omg_r**2)

# spectrum resolution criterion (should be ~1)
cry = de*r_max/np.sqrt(2*10*Upu)/np.pi
# some dipole matrix elements
d21 = 128*np.sqrt(2)/243 # <210|r|100>
d31 = 27/64/np.sqrt(2) # <310|r|100>
d32 = 110592*np.sqrt(3)/78125 # <320|r|210>
d42 = 4096*np.sqrt(2)/6561 # <420|r|210>
d43 = 191102976*np.sqrt(6/5)/40353607 # <430|r|320>

# quasi-classical near-threshold bound-free dipole matrix element
# see Fedorov "Atomic and Free Electrons in a Strong Light Field," Section 1.4c.
# approx. dipole coupling between a Rydberg state (n>>1) and near-threshold (E<<1) continuum state at E = En + omega.
def dnc(n, omega):
    return 0.22/(n**(3/2))/(omega**(5/3))

# Rabi frequencies & periods
Ru = np.sqrt((d32*Fu)**2 + (omg32-omg_u)**2)
Rr = np.sqrt((d32*Fr)**2 + (omg32-omg_r)**2)
Twu = 2*np.pi/Ru
Twr = 0.0
if (Fr != 0.0): Twr = 2*np.pi/Rr

# echo loaded parameters
print('\n'+run_path)
print('----------------------------------------')
print('domain')
print('t_max = ' + str(np.round(t_max*1e-4,4)) + 'e+4 a.u.')
print('r_max = ' + str(np.round(r_max*1e-3,4)) + 'e+3 a.u.')
print('l_max = ' + str(l_max))
print('dr = ' + str(dr) + ' a.u.')
print('----------------------------------------')
print('atomic properties')
print('nlm: (' + str(enn) + ',' + str(ell) + ',' + str(emm) + ')')
print('----------------------------------------')
print('p-electron spectrum')
print('rng: [' + str(e_min) + ', ' + str(e_max) + '] a.u.')
print('res: ' + str(np.round(de*1e+4,2)) + 'e-4 a.u.')
print('ord: ' + str(nwop))
print('cry: ' + str(np.round(cry,3)))
print('----------------------------------------')
print('field')
print('Fu = ' + str(np.round(Fu*1e+3,4)) + 'e-3 a.u., Iu = ' + str(np.round((Fu**2)*au.I*1e-13,2)) + ' GW/cm2')
print('Fr = ' + str(np.round(Fr*1e+3,4)) + 'e-3 a.u., Ir = ' + str(np.round((Fr**2)*au.I*1e-13,2)) + ' GW/cm2')
print()
print('omg_u = ' + str(omg_u) + ' a.u., λu = ' + str(np.round(2*np.pi*mks.c/(omg_u/au.t)*1e+9,2)) + ' nm')
print('omg_r = ' + str(omg_r) + ' a.u., λr = ' + str(np.round(2*np.pi*mks.c/(omg_r/au.t)*1e+9,2)) + ' nm')
print()
print('tu_on: ' + str(tu_on) + ' a.u.')
print('tr_on: ' + str(tr_on) + ' a.u.')
print('----------------------------------------\n')

### Load data

In [ ]:
f = h5py.File(home_path+'/'+run_path+'/'+'output.h5','r')

# KEY NAMES MAY DIFFER BETWEEN OUTPUT FILES
#print(list(f.keys()))

# wavefunction matrix
# its: time index skip, r_rng: radial window
load_phi = True; its = 10; r_rng = [0.0, 5.0e+3];

r = np.array(f['r'][:,0]); nr = len(r); dr = r[1]-r[0];
t = np.array(f['t'][:,0]); nt = len(t); dt = t[1]-t[0];
tf = np.array(f['tf'][:,0]); ntf = len(tf); dtf = tf[1]-tf[0];
print('loaded grid')

Eu = np.array(f['E_u(t)'][:,0]); Au = np.array(f['A_u(t)'][:,0]);
Er = np.array(f['E_r(t)'][:,0]); Ar = np.array(f['A_r(t)'][:,0]);
print('loaded fields')

no = np.array(f['norm(phi)'][:,0])
en = np.array(f['enrg(phi)'][:,0])
print('loaded norm & energy')

Pbt = np.array(f['Pb(t)']).transpose()
Pft = np.array(f['Pf(t)']).transpose()
print('loaded bound-state occupations')

if ('E(c)' in f.keys()): Ec = np.array(f['E(c)'][:,0])
if ('l(c)' in f.keys()): lc = np.array(f['l(c)'][:,0]).astype(int)

if (load_phi):
    phi = np.array(f['Re(phi)'][0::its,int(r_rng[0]/dr):int(r_rng[1]/dr),:]) \
     + 1j*np.array(f['Im(phi)'][0::its,int(r_rng[0]/dr):int(r_rng[1]/dr),:])
    psi = np.sum(phi,2) # sum over l-channels
    print('loaded wavefunction')
    print('  dim(phi) (t,r,l) = ' + str(np.shape(phi)) + ' -deleted')
    del phi

E = np.array(f['E'][:,0])
dW_dE = np.array(f['dW_dE']).transpose()
print('loaded spectrum')

# calculate bound/free occupations from normalized spectrum
ion_prob = np.zeros(ntf)
bnd_prob = np.zeros(ntf)

for k in range(ntf):
    ion_prob[k] = de * np.trapz(dW_dE[k,int(abs(E[0])/de):])
    bnd_prob[k] = de * np.trapz(dW_dE[k,0:int(abs(E[0])/de)])

# imshow windows
extent1 = [0.0, r_max, 0.0, t_max*au.t]
extent1_2 = [r_rng[0], r_rng[1], 0.0, t_max*au.t*1e+15]
extent1_3 = [r_rng[0], r_rng[1], 0.0, t_max/T0u]
extent2_au = [e_min, e_max, 0.0, t_max]
extent2_eVfs = [e_min*au.E/mks.e, e_max*au.E/mks.e, 0.0, t_max*au.t*1e+15]
extent2_eVT0 = [e_min*au.E/mks.e, e_max*au.E/mks.e, 0.0, t_max/T0u]
print('done')

### Plot settings

In [ ]:
# plot settings
font = {'size':16,'weight':'normal'} # default 'size':20
matplotlib.rc('font',**font)
dpi = 75 # 300 for production

#matplotlib.rcParams['font.family'] = 'sans-serif'
#matplotlib.rcParams['font.sans-serif'] = 'Arial'

### Plot time-dependent electron spectrum

In [ ]:
label_fs = 9
xlims = [e_min*au.E/mks.e, e_max*au.E/mks.e]
ylims = [0.0, t_max/T0u]
clims = [-3, 3]
aspect = 0.1
save_PNG = False
save_EPS = False

# plot
plt.figure(figsize=(8,6), dpi=dpi, facecolor='w')
plt.imshow(np.log10(dW_dE), extent=extent2_eVT0, origin='lower', aspect=aspect, cmap='turbo')

plt.xlabel('Electron energy (eV)'); plt.ylabel('Time ($T_0$)')

plt.xlim(xlims), plt.ylim(ylims), plt.clim(clims);
cbar = plt.colorbar(shrink=0.87, pad=0.02); cbar.set_label('Probability density, $\\log(dW/dE)$')

ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_major_locator(MultipleLocator(40))
ax.yaxis.set_minor_locator(MultipleLocator(10))

plt.tight_layout()
if (save_PNG): plt.savefig('tdpes.png', dpi=dpi)
if (save_EPS): plt.savefig('tdpes.eps', format='eps')
plt.show()

### Plot integrated wavefunction

In [ ]:
plt.figure(figsize=(8,6), dpi=dpi, facecolor='w')
plt.imshow(np.ma.log10(abs(psi)**2).filled(-100), extent=extent1_3, origin='lower', aspect=5, cmap='turbo')

plt.xlabel('Radial coordinate ($a_0$)'), plt.ylabel('Time ($T_0$)')

ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(1000))
ax.xaxis.set_minor_locator(MultipleLocator(500))
ax.yaxis.set_major_locator(MultipleLocator(40))
ax.yaxis.set_minor_locator(MultipleLocator(20))

plt.colorbar(shrink=0.8, label='$\\log$ Wavefunction, $\\sum|\\varphi_{n\ell}(\\vec{r})|^2$')
plt.clim([-6,-2])

plt.tight_layout()
#plt.savefig('psit.png', dpi=dpi)
#plt.savefig('psit.eps', format='eps')
plt.show()

### Plot bound/ionized fractions (from normalized spectrum)

In [ ]:
plt.plot(au.t*tf*1e+15, bnd_prob, label='bound', c='tab:blue')
plt.plot(au.t*tf*1e+15, ion_prob, label='ionized', c='tab:orange')
plt.xlabel('Time (fs)'), plt.ylabel('Probability'), plt.legend(frameon=False)
plt.show()

### Plot bound-state projections

In [ ]:
plt.figure(figsize=(7,5), dpi=dpi, facecolor='w')

tp = t/Twu

lw = 1.2 # linewidth
plt.plot(tp, 1e+2*Pbt[1,:], c='k', label='$2p$', lw=1)
plt.plot(tp, 1e+2*Pbt[2,:], '--', c='k', label='$3d$', lw=1)
plt.plot(tp, 1e+2*np.sum(Pbt,axis=0), c='blue', label='BOUND', lw=lw)
plt.plot(tp, 1e+2*(1.0-np.sum(Pbt,axis=0)), c='tab:red', label='FREE', lw=lw)
plt.xlabel('Time ($T_\\Omega$)'), plt.ylabel('Level occupation (%)')

''' use with tp = t*au.t
gamma = 2.5e+11;
plt.plot(tp, 1e+2*np.exp(-gamma*tp), c='g')
'''

leg = plt.legend(loc='upper center', ncol=4, fontsize=11, bbox_to_anchor=(0.5, 1.06), framealpha=1.0, fancybox=False, edgecolor='k', shadow=True)

ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_major_locator(MultipleLocator(20))
ax.yaxis.set_minor_locator(MultipleLocator(10))

plt.tight_layout()
#plt.savefig('bsp.png', dpi=dpi)
#plt.savefig('bsp.eps', format='eps')
plt.show()

### Plot continuum-state projections

In [ ]:
plt.figure(figsize=(7,5), dpi=dpi, facecolor='w')

tp = t/Twu

lw = 1.2 # linewidth
#plt.plot(tp, 1e+2*Pft[0,:], c='k', label='$|E_1 f 0\\rangle$', lw=1)
plt.plot(tp, 1e+4*Pft[1,:], '-', c='k', label='$|E_2 f 0\\rangle$', lw=1)
plt.xlabel('Time ($T_\\Omega$)'), plt.ylabel('Level occupation ($\\times10^{-2}$) (%)')

leg = plt.legend(loc='upper center', ncol=4, fontsize=11, bbox_to_anchor=(0.5, 1.06), framealpha=1.0, fancybox=False, edgecolor='k', shadow=True)

'''
ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_major_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(0.125))
'''

plt.tight_layout()
#plt.savefig('csp.png', dpi=dpi)
#plt.savefig('csp.eps', format='eps')
plt.show()

### Plot time lineouts of electron spectrum

In [ ]:
# time indices to plot
i = [0,-1]; ni = len(i);
# legend labels
lebs = ['$T_A$','$T_B$']
# colors
cs = ['mediumblue','tab:red']
# linewidth
lw = 1.5

for k in range(ni):
    print('t('+str(k)+') = ' + str(np.round(tf[i[k]]/T0u,2)) + \
          ' u. cyc. or ' + str(np.round(tf[i[k]]*au.t*1e+15,2)) + ' fs')

plt.figure(figsize=(8,6), dpi=dpi, facecolor='w')
for k in range(ni):
    plt.semilogy(E/omg_u, (10**(3*k))*dW_dE[i[k],:], lw=lw, c=cs[k], label=lebs[k])

ax = plt.gca()
ax.tick_params(labelleft=False)

plt.grid(alpha=0.6, which='both')

ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_minor_locator(MultipleLocator(1))

plt.xlabel('Electron energy ($\\omega$)', usetex=False)
plt.ylabel('Probability density, $\\log(dW/dE)$', usetex=False)

plt.text(0.05, 1e-9*0.13e1, 'CONTINUUM', fontsize=11, ha='left', va='bottom', rotation=90)

leg = plt.legend(loc='best', title='\\textbf{Time}', ncol=1, fontsize=12, framealpha=1.0, fancybox=False, edgecolor='k', shadow=True)
plt.setp(leg.get_title(), fontsize='12', usetex=True)

plt.tight_layout()
#plt.savefig('pes.png', dpi=dpi)
#plt.savefig('pes.eps', format='eps')
plt.show()

### Plot total field vs. time

In [ ]:
plt.figure(figsize=(7,5), dpi=dpi, facecolor='w')
plt.plot(t/T0u, Eu*au.F*1e-9, lw=0.3, c='darkred')
plt.plot(t/T0u, Er*au.F*1e-9, lw=0.3, c='darkblue')

plt.xlabel('Time ($T_0$)'), plt.ylabel('Field strength (GV/m)')

ax = plt.gca()
ax.xaxis.set_minor_locator(MultipleLocator(10))

plt.tight_layout()
#plt.savefig('fld.png', dpi=dpi)
#plt.savefig('fld.eps', format='eps')
plt.show()

### Plot norm / energy

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,7), dpi=dpi, facecolor='w')

ln1 = ax1.plot(t/T0u, en*au.E/mks.e, 'mediumblue', lw=2.0);
ax1.spines['left'].set_color('mediumblue');
ax1.tick_params(axis='y', colors='mediumblue');
plt.ylabel('$\\langle H(\\vec{r},\\vec{p})\\rangle$ (eV)', c='mediumblue', labelpad=10);

ax2 = ax1.twinx();
ln2 = ax2.plot(t/T0u, no, 'k', lw=2.0);
plt.ylabel('$\\int|\\Psi(\\vec{r},t)|^2\\,d^2r$', labelpad=15);

ax1.set_xlabel('Time ($T_0$)', labelpad=10);
ax1.set_xlim(np.divide([t[0],t[-1]],T0u));

ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(50))
ax.xaxis.set_minor_locator(MultipleLocator(25))

plt.tight_layout()
#plt.savefig('expvals.png', dpi=dpi)
#plt.savefig('expvals.eps', format='eps')
plt.show()